In [2]:
import pydap
import netCDF4 as nc
import pandas as pd
import h5py
import xarray as xr
import geopy.distance
import numpy as np

In [10]:
# import numpy as np
# import pandas as pd

# # Load the XCO2 data from OCO-2 and GOSAT
# df_oco2 = pd.read_csv("df_oco2.csv")
# df_gosat = pd.read_csv("df_gosat.csv")

# # Align the data in terms of time and location
# df_oco2.sort_values(by=["time", "latitude", "longitude"], inplace=True)
# df_gosat.sort_values(by=["time", "latitude", "longitude"], inplace=True)

# # Merge the data by taking the average of the XCO2 values for a given location and time
# combined_data = pd.DataFrame(columns=["time", "latitude", "longitude", "xco2"])
# oco2_index = 0
# gosat_index = 0
# while oco2_index < len(df_oco2) and gosat_index < len(df_gosat):
#     oco2_time = df_oco2.iloc[oco2_index]["time"]
#     gosat_time = df_gosat.iloc[gosat_index]["time"]
#     if oco2_time < gosat_time:
#         oco2_index += 1
#     elif oco2_time > gosat_time:
#         gosat_index += 1
#     else:
#         oco2_latitude = df_oco2.iloc[oco2_index]["latitude"]
#         gosat_latitude = df_gosat.iloc[gosat_index]["latitude"]
#         oco2_longitude = df_oco2.iloc[oco2_index]["longitude"]
#         gosat_longitude = df_gosat.iloc[gosat_index]["longitude"]
#         if abs(oco2_latitude - gosat_latitude) < 1e-6 and abs(oco2_longitude - gosat_longitude) < 1e-6:
#             combined_xco2 = (df_oco2.iloc[oco2_index]["xco2"] + df_gosat.iloc[gosat_index]["xco2"]) / 2
#             combined_data = combined_data.append({"time": oco2_time, "latitude": oco2_latitude, "longitude": oco2_longitude, "xco2": combined_xco2}, ignore_index=True)
#             oco2_index += 1
#             gosat_index += 1

# # Save the combined data to a CSV file
# combined_data.to_csv("combined_data.csv", index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'df_oco2.csv'

In [3]:
f_gosat = nc.Dataset("gosat/2016/acos_LtCO2_160102_v201201_B7310A_161107211938s.nc4")
f_oco2 = nc.Dataset("oco2_2016_jan1.nc4")

In [4]:
f_gosat

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    filter_function: acos_l2_filter_b7
    bc_function: bias_correct_acos_b7
    warn_level_function: acos_warn_levels_b7
    Bias_Correction_landH: XCO2_Bias_Corrected = XCO2_Raw + 0.300*(dp + 0.0) + 8.60*(sqrt_alb_3 - 0.500) + 0.0160*(co2_grad_del - 25.0) + 14.5*(DWS - 0.0200) +0.15 ppm
    Bias_Correction_landM: XCO2_Bias_Corrected = XCO2_Raw + 0.290*(dp + 1.20) + 4.10*(alb_3 - 0.400) + 0.0150*(co2_grad_del - 20.0) + 7.50*(aod_du - 0.0100) -0.20 ppm
    Bias_Correction_Ocean: XCO2_Bias_Corrected = XCO2_Raw - 42.4*(s32 - 0.610) - 0.0930*(co2_grad_del + 3.00) + 1.80*(aodheight_ic - 0.180) + 0.325*(logDust + 0.0) +0.90 ppm
    Bias_Uncertainty: Bias correction parameter uncertainties are given in the data users guide.
    Note_regarding_bias_correction: There is some uncertainty in not only the bias correction coefficients but in the chosen parameters themselves. This release bias correction may be sl

In [5]:
f_gosat.variables.keys()

dict_keys(['sounding_id', 'levels', 'bands', 'date', 'latitude', 'longitude', 'time', 'solar_zenith_angle', 'sensor_zenith_angle', 'xco2_quality_flag', 'source_files', 'file_index', 'warn_level', 'xco2', 'xco2_uncertainty', 'xco2_apriori', 'pressure_levels', 'co2_profile_apriori', 'xco2_averaging_kernel', 'pressure_weight'])

In [6]:
f_gosat['date']

<class 'netCDF4._netCDF4.Variable'>
int16 date(sounding_id, epoch_dimension)
    units: none
    long_name: Observation date and time matching sounding_id
    missing_value: -9999
    comment: Year, month (1-12), day (1-31), hour (0-23), minute (0-59), second (0-59).
unlimited dimensions: 
current shape = (735, 6)
filling on, default _FillValue of -32767 used

In [7]:
# date format:Year, month (1-12), day (1-31), hour (0-23), minute (0-59), second (0-59).
dates =[]
for dt in f_gosat['date']:
    s = ""
    for num in dt:
        s += str(num)+"/"
    dates.append(s[:-1])

In [11]:
df_oco2 = pd.DataFrame(columns=["latitude","longitude","time",'sensor_zenith_angle', 'xco2', 'xco2_uncertainty', 'xco2_apriori' ])
df_gosat = pd.DataFrame(columns=df_oco2.columns)

In [12]:
for col in df_gosat.columns:
    print(col)
    df_gosat[col] = [x for x in f_gosat[col]]

latitude
longitude
time
sensor_zenith_angle
xco2
xco2_uncertainty
xco2_apriori


In [13]:
len(dates)

735

In [14]:
df_gosat["dates"] = dates
df_gosat["dates"] = pd.to_datetime(df_gosat["dates"],format='%Y/%m/%d/%H/%M/%S')

In [15]:
df_gosat.shape

(735, 8)

In [16]:
df_gosat.head()

,latitude,longitude,time,sensor_zenith_angle,xco2,xco2_uncertainty,xco2_apriori,dates
0,-24.553658,172.7425,1451695945.2100575,24.090277,398.15314,1.3915493,399.66754,2016-01-02 00:52:23
1,-24.547958,172.74211,1451695949.9356806,24.628723,399.34375,1.419487,399.66754,2016-01-02 00:52:28
2,-24.544949,172.7403,1451695954.4291587,25.432869,399.25946,1.4963825,399.66757,2016-01-02 00:52:32
3,-77.64792,117.06473,1451696871.832189,26.95575,404.99014,3.3346617,398.85254,2016-01-02 01:07:50
4,-83.17064,120.690865,1451696917.927493,25.893877,409.90796,4.2980876,398.57144,2016-01-02 01:08:36


In [17]:
df_gosat.iloc[0]['latitude']

masked_array(data=-24.553658,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [18]:
df_gosat.describe()

/var/folders/wx/g95tcl_55cg_v5h5q3hkngmh0000gn/T/ipykernel_18165/10778488.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_gosat.describe()


,latitude,longitude,time,sensor_zenith_angle,xco2,xco2_uncertainty,xco2_apriori,dates
count,735,735,735,735,735,735,735,735
unique,733,735,735,735,735,735,698,735
top,-84.34673,172.7425,1451695945.2100575,24.090277,398.15314,1.3915493,399.3372,2016-01-02 00:52:23
freq,2,1,1,1,1,1,3,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-02 00:52:23
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-02 23:35:39


In [21]:
for col in df_oco2.columns:
    print(col)
    df_oco2[col] = [x for x in f_oco2[col]]

latitude
longitude
time
sensor_zenith_angle
xco2
xco2_uncertainty
xco2_apriori


In [20]:
combined_data = pd.DataFrame(columns=["time", "latitude", "longitude", "xco2"])
oco2_index = 0
gosat_index = 0
diff1 =[]
diff2 =[]
while oco2_index < len(df_oco2):
    oco2_latitude = df_oco2.iloc[oco2_index]["latitude"]
    oco2_longitude = df_oco2.iloc[oco2_index]["longitude"]
    while gosat_index < len(df_gosat):
        gosat_latitude = df_gosat.iloc[gosat_index]["latitude"]
        gosat_longitude = df_gosat.iloc[gosat_index]["longitude"]
        diff1.append(abs(oco2_latitude - gosat_latitude))
        diff2.append(abs(oco2_longitude - gosat_longitude))
        if abs(oco2_latitude - gosat_latitude) < 1 and abs(oco2_longitude - gosat_longitude) < 1:
            combined_xco2 = (df_oco2.iloc[oco2_index]["xco2"] + df_gosat.iloc[gosat_index]["xco2"]) / 2
            combined_data = combined_data.append({"time": oco2_time, "latitude": oco2_latitude, "longitude": oco2_longitude, "xco2": combined_xco2},ignore_index=True)
        gosat_index += 1
    oco2_index += 1
    

# Save the combined data to a CSV file
combined_data.to_csv("combined_data.csv", index=False)
print(min(diff1))
print(min(diff2))


0.7831883430480957
2.7499237060546875


In [21]:
combined_data

,time,latitude,longitude,xco2


In [20]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print (geopy.distance.geodesic(coords_1, coords_2).km)

279.35290160430094
